In [20]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd


list_of_cities = ['seattle',  'miami']

# Load neighborhood polygons from GeoJSON
neighborhoods = gpd.read_file('data/seattle/neighbourhoods.geojson')

# Check the column names in neighborhoods GeoDataFrame
print("Columns in neighborhoods GeoDataFrame:", neighborhoods.columns)

# Identify the column that contains neighborhood names
# For example, if the column name is 'neighborhood'
neighborhood_name_column = 'neighbourhood'  

# Define a function to process each chunk
def process_chunk(chunk):
    # Convert latitude-longitude to a GeoDataFrame
    geometry = [Point(xy) for xy in zip(chunk.lng, chunk.lat)]
    latlong_gdf = gpd.GeoDataFrame(chunk, crs="EPSG:4326", geometry=geometry)

    # Perform the spatial join
    joined_data = gpd.sjoin(latlong_gdf, neighborhoods, how="left", predicate="within")

    # Keep only the neighborhood name column from the joined data
    chunk['neighborhood'] = joined_data[neighborhood_name_column]

    return chunk

# Initialize an empty DataFrame to hold the processed data
processed_data = pd.DataFrame()

# Read the CSV in chunks
chunksize = 10000  # Adjust the chunksize based on your system's memory capacity
for chunk in pd.read_csv('data/city_stats/uszips.csv.gz', chunksize=chunksize):
    processed_chunk = process_chunk(chunk)
    processed_data = pd.concat([processed_data, processed_chunk])

# Save the processed data to a new CSV file
processed_data.to_csv('data/city_stats/enriched_uszips_with_neighborhoods.csv')


Columns in neighborhoods GeoDataFrame: Index(['neighbourhood', 'neighbourhood_group', 'geometry'], dtype='object')


/var/folders/bj/6vmxzyc10bl0w0z6d16ghc9c0000gn/T/ipykernel_6656/2430036244.py:34: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('data/city_stats/uszips.csv.gz', chunksize=chunksize):


In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from config import *

def tag_neighborhoods(list_of_cities, include_na=True):
    processed_data = pd.DataFrame()

    for city in list_of_cities:
        # Load neighborhood 
        neighborhoods = gpd.read_file(f'data/{city}/neighbourhoods.geojson')
        


        # Define a function to process each chunk
        def process_chunk(chunk):
            # Convert latitude-longitude to a GeoDataFrame
            geometry = [Point(xy) for xy in zip(chunk.lng, chunk.lat)]
            latlong_gdf = gpd.GeoDataFrame(chunk, crs="EPSG:4326", geometry=geometry)

            # Join
            joined_data = gpd.sjoin(latlong_gdf, neighborhoods, how="left", predicate="within")

            # Keep only the neighborhood name column from the joined data
            chunk['neighborhood'] = joined_data['neighbourhood']

            return chunk

        # Read the CSV in chunks and process
        chunksize = 10000  
        for chunk in pd.read_csv(f'data/city_stats/uszips.csv.gz', chunksize=chunksize):
            processed_chunk = process_chunk(chunk)
            # Filter based on include_na flag
            if not include_na:
                processed_chunk = processed_chunk.dropna(subset=['neighborhood'])
            processed_data = pd.concat([processed_data, processed_chunk])

    return processed_data

In [2]:
df = tag_neighborhoods(list_of_cities, include_na=True)

/var/folders/bj/6vmxzyc10bl0w0z6d16ghc9c0000gn/T/ipykernel_22286/2334401059.py:32: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f'data/city_stats/uszips.csv.gz', chunksize=chunksize):
/var/folders/bj/6vmxzyc10bl0w0z6d16ghc9c0000gn/T/ipykernel_22286/2334401059.py:32: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f'data/city_stats/uszips.csv.gz', chunksize=chunksize):
/var/folders/bj/6vmxzyc10bl0w0z6d16ghc9c0000gn/T/ipykernel_22286/2334401059.py:32: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f'data/city_stats/uszips.csv.gz', chunksize=chunksize):
/var/folders/bj/6vmxzyc10bl0w0z6d16ghc9c0000gn/T/ipykernel_22286/2334401059.py:32: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in 